In [1]:
import pandas as pd

In [2]:
binding = pd.read_csv('/mnt/nas/ai-project-data/pingping/concat/BindingDB_All.tsv', sep = '\t', error_bad_lines=False)

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 1087314: expected 193 fields, saw 205\nSkipping line 1087340: expected 193 fields, saw 205\n'
b'Skipping line 1120033: expected 193 fields, saw 205\n'
b'Skipping line 1142701: expected 193 fields, saw 265\n'
b'Skipping line 1546347: expected 193 fields, saw 241\n'
b'Skipping line 1660234: expected 193 fields, saw 241\nSkipping line 1660235: expected 193 fields, saw 241\nSkipping line 1660236: expected 193 fields, saw 241\nSkipping line 1660237: expected 193 fields, saw 241\nSkipping line 1660238: expected 193 fields, saw 241\nSkipping line 1660239: expected 193 fields, saw 241\nSkipping line 1660240: expected 193 fields, saw 241\nSkipping line 1660241: expected 193 fields, saw 241\nSkipping line 1660242: expected 

In [3]:
binding.columns

Index(['BindingDB Reactant_set_id', 'Ligand SMILES', 'Ligand InChI',
       'Ligand InChI Key', 'BindingDB MonomerID', 'BindingDB Ligand Name',
       'Target Name Assigned by Curator or DataSource',
       'Target Source Organism According to Curator or DataSource', 'Ki (nM)',
       'IC50 (nM)',
       ...
       'UniProt (SwissProt) Recommended Name of Target Chain.12',
       'UniProt (SwissProt) Entry Name of Target Chain.12',
       'UniProt (SwissProt) Primary ID of Target Chain.12',
       'UniProt (SwissProt) Secondary ID(s) of Target Chain.12',
       'UniProt (SwissProt) Alternative ID(s) of Target Chain.12',
       'UniProt (TrEMBL) Submitted Name of Target Chain.12',
       'UniProt (TrEMBL) Entry Name of Target Chain.12',
       'UniProt (TrEMBL) Primary ID of Target Chain.12',
       'UniProt (TrEMBL) Secondary ID(s) of Target Chain.12',
       'UniProt (TrEMBL) Alternative ID(s) of Target Chain.12'],
      dtype='object', length=193)

In [4]:
pd.set_option('display.max_rows',10)      #设置最大行数
pd.set_option('display.max_columns', None)  #设置最大列数

In [5]:
binding.shape

(2407235, 193)

In [6]:
usebinding = binding[['Ligand SMILES', 'IC50 (nM)', 'Ki (nM)', 'Kd (nM)', 'BindingDB Target Chain  Sequence', 'UniProt (SwissProt) Primary ID of Target Chain', 'Target Source Organism According to Curator or DataSource', 'Article DOI']]

In [7]:
usebinding.shape

(2407235, 8)

In [8]:
usebinding.rename(columns={"Ligand SMILES": "SMILES", "Ligand InChI Key": "InChI_Key", "IC50 (nM)": "IC50_nM", \
                           "Ki (nM)": "Ki_nM", "Kd (nM)": "Kd_nM", "BindingDB Target Chain  Sequence": "Target_Chain_Sequence", \
                           "UniProt (SwissProt) Primary ID of Target Chain": "Uniprot_ID", 'Target Source Organism According to Curator or DataSource': 'Target Source', 'Article DOI': 'DOI'}, inplace = True)

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
usebinding.isnull().sum()

SMILES                         5
IC50_nM                   846412
Ki_nM                    1878041
Kd_nM                    2311683
Target_Chain_Sequence        575
Uniprot_ID                229004
Target Source             793106
DOI                       967166
dtype: int64

In [10]:
usebinding.to_csv('bindingdb2022_raw.csv', index = False)

In [11]:
ic50 = usebinding.dropna(subset = ['SMILES', 'IC50_nM', 'Target_Chain_Sequence', 'Uniprot_ID'])[['SMILES', 'IC50_nM', 'Target_Chain_Sequence', 'Uniprot_ID', 'Target Source', 'DOI']]

In [12]:
ic50.shape

(1390819, 6)

In [13]:
ic50 = ic50[~ic50['SMILES'].str.contains('\.' or '\t' or '\r' or '\n')]
ic50.shape

(1381636, 6)

In [14]:
def num(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return "Error"

In [15]:
ic50['transfer'] = ic50['IC50_nM'].apply(lambda x : num(x))

In [16]:
ic50[ic50['transfer']=='Error']

,SMILES,IC50_nM,Target_Chain_Sequence,Uniprot_ID,Target Source,DOI,transfer
142,Cc1nc(CN2CCN(CC2)c2c(Cl)cnc3[nH]c(nc23)-c2cn(C...,>50000,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,P08684,Homo sapiens,NaN,Error
1898,C[C@@H]1CC[C@H](CN1C(=O)c1ccc(F)cc1-n1nccn1)Oc...,>10000,MEPSATPGAQPGVPTSSGEPFHLPPDYEDEFLRYLWRDYLYPKQYE...,P56718,Rattus norvegicus,NaN,Error
3132,CN1C(=O)C(=C(C1=O)c1c[nH]c2ccccc12)c1c[nH]c2cc...,>100000,MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQ...,P68403,Rattus norvegicus,10.1021/jm00079a024,Error
3134,O=C1NC(=O)[C@H]([C@@H]1c1c[nH]c2ccccc12)c1c[nH...,>100000,MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQ...,P68403,Rattus norvegicus,10.1021/jm00079a024,Error
3135,O=C1NC(=O)[C@@H]([C@@H]1c1c[nH]c2ccccc12)c1c[n...,>100000,MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQ...,P68403,Rattus norvegicus,10.1021/jm00079a024,Error
...,...,...,...,...,...,...,...
2407155,CC(C)NC[C@H](O)COc1ccc(cc1)\N=N\c1ccc(NC(C)=O)cc1,>2000,MGQPGNGSAFLLAPNGSHAPDHDVTQERDEVWVVGMGIVMSLIVLA...,P07550,Homo sapiens,10.1021/acs.jmedchem.0c00831,Error
2407159,CC(C)NC[C@H](O)COc1ccc(cc1)\N=N\c1ccc(NC(C)=O)cc1,>2000,MGQPGNGSAFLLAPNGSHAPDHDVTQERDEVWVVGMGIVMSLIVLA...,P07550,Homo sapiens,10.1021/acs.jmedchem.0c00831,Error
2407220,COc1cc(\C=C2/CCCN3[C@@H](C)[C@H](ON=C23)c2cc(F...,>10000,MEVRPKESWNHADFVHCEDTESVPGKPSVNADEEVGGPQICRVCGD...,O75469,NaN,10.1021/acs.jmedchem.0c00446,Error
2407221,COc1cc(ccc1-n1cnc(C)c1)C1=CC2=NO[C@@H](CN2CCC1...,>30000,MEVRPKESWNHADFVHCEDTESVPGKPSVNADEEVGGPQICRVCGD...,O75469,NaN,10.1021/acs.jmedchem.0c00446,Error


In [17]:
filter_ic50 = ic50[ic50['transfer'] != 'Error']

In [18]:
filter_ic50 = filter_ic50[['SMILES', 'IC50_nM', 'Target_Chain_Sequence', 'Uniprot_ID', 'Target Source', 'DOI']]

In [19]:
filter_ic50.shape

(1108359, 6)

In [20]:
filter_ic50.isnull().sum()

SMILES                        0
IC50_nM                       0
Target_Chain_Sequence         0
Uniprot_ID                    0
Target Source            351132
DOI                      469936
dtype: int64

In [21]:
from rdkit import Chem
from openbabel import pybel
def smiles2inchi(smi):
    """
    Smiles to Inchi_key.

    Args:
        smi (str): smiles.

    Returns:
        [str]: Inchikey.
    """
    try:
        mol = Chem.MolFromSmiles(smi)
        inchi = Chem.inchi.MolToInchiKey(mol)
    except:
        inchi = np.nan
    return inchi

def smiles2inchi2(smi):
    try:
        mol = Chem.MolFromSmiles(smi)
        inchi = Chem.inchi.MolToInchiKey(mol)
    except:
        inchi = "Error" 
    return inchi

def smiles_unique(smiles):
    """
    Smiles to Cnao_smiles

    Args:
        smiles (str): Smiles.

    Returns:
        [str]: Cano_smiles.
    """
    try:
        cans = [pybel.readstring("smi", smile).write("can").strip('\t\n') for smile in smiles]
    except:
        cans = np.nan
    return cans

In [22]:
filter_ic50['gxl_inchi'] = filter_ic50['SMILES'].apply(lambda x : smiles2inchi2(x))

RDKit ERROR: [12:08:50] Explicit valence for atom # 28 N, 4, is greater than permitted
[12:08:50] Explicit valence for atom # 28 N, 4, is greater than permitted
RDKit ERROR: [12:08:54] Explicit valence for atom # 12 N, 4, is greater than permitted
[12:08:54] Explicit valence for atom # 12 N, 4, is greater than permitted
RDKit ERROR: [12:08:54] Explicit valence for atom # 12 N, 4, is greater than permitted
[12:08:54] Explicit valence for atom # 12 N, 4, is greater than permitted
RDKit ERROR: [12:09:00] Explicit valence for atom # 16 O, 3, is greater than permitted
[12:09:00] Explicit valence for atom # 16 O, 3, is greater than permitted
RDKit ERROR: [12:09:00] Explicit valence for atom # 16 O, 3, is greater than permitted
RDKit ERROR: [12:09:00] Explicit valence for atom # 17 O, 3, is greater than permitted
RDKit ERROR: [12:09:00] Explicit valence for atom # 16 O, 3, is greater than permitted
RDKit ERROR: [12:09:00] Explicit valence for atom # 16 O, 3, is greater than permitted
[12:09:0

In [23]:
uniprot_filter = filter_ic50[(filter_ic50['gxl_inchi'] != 'Error')] 

In [24]:
uniprot_filter.shape

(1107329, 7)

In [25]:
uniprot_filter.isnull().sum()

SMILES                        0
IC50_nM                       0
Target_Chain_Sequence         0
Uniprot_ID                    0
Target Source            350742
DOI                      468970
gxl_inchi                     0
dtype: int64

In [26]:
len(uniprot_filter.gxl_inchi.unique())

596142

In [27]:
uniprot_filter['IC50_nM_num'] = uniprot_filter['IC50_nM'].apply(lambda x: num(x))

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
import numpy as np
uniprot_filter['IC50pki'] = uniprot_filter['IC50_nM_num'].apply(lambda x: round(-np.log10(x * 10**(-9)), 4))

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
uniprot_filter.shape

(1107329, 9)

In [30]:
uniprot_filter.isnull().sum()

SMILES                        0
IC50_nM                       0
Target_Chain_Sequence         0
Uniprot_ID                    0
Target Source            350742
DOI                      468970
gxl_inchi                     0
IC50_nM_num                   0
IC50pki                       0
dtype: int64

In [31]:
uniprot_filter.loc[:, "COMPOUND_SMILES"] = uniprot_filter['SMILES'].apply(lambda x: smiles_unique([x])[0])
uniprot_filter.loc[:, "INCHI_KEY"] = uniprot_filter['COMPOUND_SMILES'].apply(lambda x: smiles2inchi(x))

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
RDKit ERROR: [12:25:28] Invalid InChI prefix in generating InChI Key
[12:25:28] Invalid InChI prefix in generating InChI Key
RDKit ERROR: [12:25:28] Invalid InChI prefix in generating InChI Key
RDKit ERROR: [12:25:28] Invalid InChI prefix in generating InChI Key
[12:25:28] Invalid InChI prefix in generating InChI Key
[12:25:28] Invalid InChI prefix in generating InChI Key
RDKit ERROR: [12:26:06] Invalid InChI prefix in generating InChI Key
[12:26:06] Invalid InChI prefix in generating InChI Key
RDKit ERROR: [12:26:07] Invalid InChI prefix in generating InChI Key
[12:26:07] Invalid InChI p

In [32]:
#删除smiles，inchi为空的数据
uniprot_filter.dropna(subset=['COMPOUND_SMILES','INCHI_KEY'],inplace=True)
uniprot_filter.shape

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(1107329, 11)

In [33]:
#处理smiles带点的数据
def remove_solution(smi):
    if '.' in smi:
        par1 = smi.split('.')[0]
        par2 = smi.split('.')[1]
        if len(par1) >= len(par2):
            return par1
        else:
            return par2
    else: 
        return smi
uniprot_filter['COMPOUND_SMILES'] = uniprot_filter['COMPOUND_SMILES'].apply(remove_solution)
print(uniprot_filter[uniprot_filter['COMPOUND_SMILES'].str.contains('\.')].shape)
print(uniprot_filter.shape)

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


(0, 11)
(1107329, 11)


In [34]:
#处理smiles带\t的数据
def remove_solution1(smi):
    if '\t' in smi:
        par1 = smi.split('\t')[0]
        par2 = smi.split('\t')[1]
        if len(par1) >= len(par2):
            return par1
        else:
            return par2
    else: 
        return smi
uniprot_filter['COMPOUND_SMILES'] = uniprot_filter['COMPOUND_SMILES'].apply(remove_solution1)
print(uniprot_filter[uniprot_filter['COMPOUND_SMILES'].str.contains('\t')].shape)
print(uniprot_filter.shape)

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


(0, 11)
(1107329, 11)


In [35]:
uniprot_filter.isnull().sum()

SMILES                        0
IC50_nM                       0
Target_Chain_Sequence         0
Uniprot_ID                    0
Target Source            350742
                          ...  
gxl_inchi                     0
IC50_nM_num                   0
IC50pki                       0
COMPOUND_SMILES               0
INCHI_KEY                     0
Length: 11, dtype: int64

In [36]:
uniprot_filter['UID'] = uniprot_filter['COMPOUND_SMILES'] + '_' + uniprot_filter['Uniprot_ID']

/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
g = uniprot_filter.groupby(['UID'], as_index=True)
value_list = g.apply(lambda x: x['IC50pki'].values)
out_pka = g['IC50pki'].count().reset_index()
out_pka['values'] = list(value_list)
print("drop duplication shape: ", out_pka.shape)

drop duplication shape:  (865492, 3)


In [38]:
out_pka

,UID,IC50pki,values
0,*Oc1ccc(cc1)n1nc(cc1NC(=O)Nc1ccc(c2c1cccc2)Oc1...,1,[6.762]
1,*Oc1ccc(cc1)n1nc(cc1NC(=O)Nc1ccc(c2c1cccc2)Oc1...,3,"[5.6031, 5.6031, 5.6031]"
2,*Oc1ccc(cc1)n1nc(cc1NC(=O)Nc1ccc(c2c1cccc2)Oc1...,3,"[5.823, 5.823, 5.823]"
3,*Oc1ccc(cc1)n1nc(cc1NC(=O)Nc1ccc(c2c1cccc2)Oc1...,2,"[6.762, 6.762]"
4,Br/C(=C/c1ccccc1)/C=NNC(=O)c1cccs1_P42574,1,[4.5003]
...,...,...,...
865487,s1[nH]c2c(n1)c1nc(nc1cc2)C1CC1_Q9Y6Q9,2,"[5.7894, 5.9219]"
865488,s1ccc(c1)c1cnc2n1cc(nc2)c1ccco1_P06241,1,[5.2041]
865489,s1ccc(c1)c1cnc2n1cc(nc2)c1cccs1_P06241,1,[5.648]
865490,s1ccc(c1)c1nc(NC2Cc3c(C2)cccc3)c2c(n1)cccc2_P0...,10,"[8.284, 8.0506, 8.284, 8.1805, 8.0506, 8.284, ..."


In [39]:
out_pka['check'] = out_pka['values'].apply(lambda x: np.ptp(x))
df_ddp_mapper = uniprot_filter[['COMPOUND_SMILES', 'UID', 'Target_Chain_Sequence', 'Uniprot_ID', 'INCHI_KEY', 'Target Source', 'DOI']].drop_duplicates(subset='UID', keep='first')
print("df_ddp_mapper shape: ", df_ddp_mapper.shape)

df_ddp_mapper shape:  (865492, 7)


In [40]:
def out_finalProcessor(out_final): 
    print("start with {}".format(out_final.shape))
    out_final = out_final[out_final['check'] < 1.3]
    print("end with {}".format(out_final.shape))
    out_final['REG_LABEL'] = out_final['values'].apply(lambda x: np.median(x))
#     out_final['CLF_GE_6_LABEL'] = out_final['REG_LABEL'].apply(lambda x: 1 if x >= 6 else 0)
#     out_final['CLF_GE_6-5_LABEL'] = out_final['REG_LABEL'].apply(lambda x: 1 if x >= 6.5 else 0)
#     out_final['CLF_GE_7_LABEL'] = out_final['REG_LABEL'].apply(lambda x: 1 if x >= 7 else 0)
    return out_final

In [41]:
final_pka = out_finalProcessor(out_pka)

start with (865492, 4)
end with (843653, 4)


/home/pingping/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [42]:
final_save = pd.merge(final_pka, df_ddp_mapper, how='left', on='UID')

In [43]:
final_save['check'] = final_save['INCHI_KEY'].apply(lambda x : len(x))

In [44]:
final_save.check.value_counts()

27    843648
0          5
Name: check, dtype: int64

In [45]:
final_save_doc = final_save[final_save['check'] > 5]

In [46]:
# 0 <= REG_LABEL <= 13
final_save_doc = final_save_doc[(final_save_doc['REG_LABEL'] >= 0)&(final_save_doc['REG_LABEL'] <= 13)]
final_save_doc.shape

(843367, 11)

In [47]:
final_save_doc = final_save_doc.drop(['IC50pki', 'check'], axis = 1)

In [48]:
final_save_doc.rename(columns={"Uniprot_ID": "UNIPROT_ID", "Target_Chain_Sequence": "PROTEIN_SEQUENCE"}, inplace = True)

In [49]:
final_save_doc.to_csv('Bindingdb_IC50_cleaned_new.csv', index=False)

In [50]:
final_save_doc.shape

(843367, 9)

In [51]:
final_save_doc.columns

Index(['UID', 'values', 'REG_LABEL', 'COMPOUND_SMILES', 'PROTEIN_SEQUENCE',
       'UNIPROT_ID', 'INCHI_KEY', 'Target Source', 'DOI'],
      dtype='object')

In [52]:
# check \t
final_save_doc[final_save_doc['COMPOUND_SMILES'].str.contains('\t')]

,UID,values,REG_LABEL,COMPOUND_SMILES,PROTEIN_SEQUENCE,UNIPROT_ID,INCHI_KEY,Target Source,DOI


In [53]:
df_human = pd.read_csv('/mnt/nas/ai-project-data/pingping/concat/Uniprot_human.csv')

In [54]:
df_human.rename(columns={'Entry': 'UNIPROT_ID'}, inplace=True)
df_human.columns

Index(['target_id', 'Sequence'], dtype='object')

In [58]:
df_human=df_human.rename(columns={'target_id':'UNIPROT_ID'})

In [59]:
out_human = final_save_doc.merge(df_human, on='UNIPROT_ID')

In [60]:
out_human.shape
out_human.to_csv('Bindingdb_IC50_human_cleaned_new.csv', index=False)

In [61]:
out_human.shape

(703746, 10)